In [18]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [19]:
# prepare data
bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target
n_samples, n_features = X.shape
print(n_samples, n_features)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# scaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

569 30


In [20]:
# model 
class LogisticRegression(nn.Module):
    def __init__(self, n_input_features):
        super(LogisticRegression, self).__init__()
        
        self.linear = nn.Linear(n_input_features, 1)
        
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred

model = LogisticRegression(n_features)

In [21]:
# loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [22]:
# training loop
num_epochs = 200
for epoch in range(num_epochs):
    # forward pass and loss
    y_pred = model(X_train)
    loss = criterion(y_pred, y_train)
    
    # backward pass
    loss.backward()
    
    # update
    optimizer.step()
    
    # zero gradients
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {loss.item():.4f}')
        
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc*100:.2f}%')

epoch: 10, loss = 0.4844
epoch: 20, loss = 0.4115
epoch: 30, loss = 0.3625
epoch: 40, loss = 0.3273
epoch: 50, loss = 0.3006
epoch: 60, loss = 0.2796
epoch: 70, loss = 0.2626
epoch: 80, loss = 0.2485
epoch: 90, loss = 0.2366
epoch: 100, loss = 0.2263
epoch: 110, loss = 0.2173
epoch: 120, loss = 0.2094
epoch: 130, loss = 0.2023
epoch: 140, loss = 0.1960
epoch: 150, loss = 0.1903
epoch: 160, loss = 0.1851
epoch: 170, loss = 0.1803
epoch: 180, loss = 0.1759
epoch: 190, loss = 0.1719
epoch: 200, loss = 0.1681
accuracy = 92.11%
